In [126]:
%load_ext autoreload 
%autoreload 

%reload_ext autoreload

import cv2 
import numpy as np 
from week1 import  PiInv, Pi,project_points,plot3d
from function_4 import * 




f = 1000 
dx = 1920/2
dy = 1080 /2 

R = np.array([[np.sqrt(0.5),-np.sqrt(0.5),0],[np.sqrt(0.5),np.sqrt(0.5),0],[0,0,1]])
t = np.array([[0],[0],[10]])


K_true = np.array([[f,0,dx],[0,f,dy],[0,0,1]])

Q = np.array([[0,0,0],[0,0,1],[0,1,1],[1,1,1],[0,1,0],[1,1,0],[1,0,1],[1,0,0]])


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Exercise 4.1
* Find the projection matrix P and the projections q.

In [127]:
P = K @ np.hstack((R,t)) 

q_i = np.array([Pi(P @ PiInv(q) ) for q in Q]).T


## Exercise 4.2 

Write a function `pest` that uses $Q$ and $q$ to estimate $P$ with the DLT. Do not normalize your points. Use the estimated projection matrix $P_{est}$ to project the points $Q$, giving you the reprojected points $q_{est}$.

What is the overall reprojection error $qP_1^n ||q_{est} - q||_2^2$ (RMSE)?

Does normalizing the 2D points before estimating $P$ (like we did for the homography) improve the reprojection error?


In [128]:
P_est = pest(PiInv(Q.T),q_i)


### The estimated P is different in scale from P_True, We thus need to find the scaling factor 
##### This can be achived either by solving the equation: $$ S \cdot P_{est_i,j} = P_{true_i,j} $$

##### This becomes  $$ S = \frac{P_{true_i,j} }{P_{est_i,j}} $$

##### Or by finding the norm of the true projection Matrix 

##### Thus our estimated matrix becomes: $$ \Large P_est = P_est \cdot S $$ 

In [129]:
S = - np.linalg.norm(P) # Remember to check for the sign 
#P_est = S* P_est 



In [130]:
q_hat = Pi(P_est @ PiInv(Q.T))

rmse = RMSE(q_hat,q_i) # Root mean square error 
print("The RMSE is: {}".format(rmse))

The RMSE is: 3.764851283320554e-16


#### Exercise 4.3
Define a function `checkerboard_points(n, m)` that returns the 3D points

$$ Q_{ij} = \begin{bmatrix} i - \frac{n-1}{2} \\ j - \frac{m-1}{2} \\ 0 \end{bmatrix} $$

where $i \in \{0, . . . , n - 1\}$ and $j \in \{0, . . . , m - 1\}$. The points should be returned as a $3 \times (n \cdot m)$ matrix and their order does not matter. These points lie in the $z = 0$ plane by definition.


In [131]:
def checkerboard_points(n, m):
    def Q_ij(i, j):
        return np.array([i - (n-1)/2, j - (m-1)/2, 0])
    
    points = np.zeros((3, n*m))
    idx = 0
    for i in range(n):
        for j in range(m):
            points[:, idx] = Q_ij(i, j)
            idx += 1
    
    return points

Exercise 4.4

Let $Q_{\Omega}$ define a set of corners on a checkerboard. Then define three sets of checkerboard points $Q_a$, $Q_b$, and $Q_c$, where

$$
Q_a = R_{\pi/10, 0, 0} \cdot Q_{\Omega}, \tag{8}
$$

$$
Q_b = R(0, 0, 0) \cdot Q_{\Omega}, \tag{9}
$$

$$
Q_c = R_{-\pi/10, 0, 0} \cdot Q_{\Omega}, \tag{10}
$$

where

$$
R(\theta_x, \theta_y, \theta_z) = \begin{bmatrix} 
\cos(\theta_z) & -\sin(\theta_z) & 0 \\
\sin(\theta_z) & \cos(\theta_z) & 0 \\
0 & 0 & 1
\end{bmatrix} \cdot 
\begin{bmatrix} 
\cos(\theta_y) & 0 & \sin(\theta_y) \\
0 & 1 & 0 \\
-\sin(\theta_y) & 0 & \cos(\theta_y)
\end{bmatrix} \cdot 
\begin{bmatrix} 
1 & 0 & 0 \\
0 & \cos(\theta_x) & -\sin(\theta_x) \\
0 & \sin(\theta_x) & \cos(\theta_x)
\end{bmatrix}. \tag{12}
$$

Recall that you can compute $R$ with scipy as follows:

```python
from scipy.spatial.transform import Rotation
R = Rotation.from_euler('xyz', [θx, θy, θz]).as_matrix()


In [132]:
from scipy.spatial.transform import Rotation


Ra = Rotation.from_euler('xyz', [np.pi/10, 0, 0]).as_matrix()
Rb = Rotation.from_euler('xyz', [0, 0, 0]).as_matrix()
Rc = Rotation.from_euler('xyz', [-np.pi/10, 0, 0]).as_matrix()

In [133]:
Q_omega = checkerboard_points(10,20)

Q_a = Ra @ Q_omega 
Q_b = Rb @ Q_omega
Q_c = Rc @ Q_omega

q_a = (P @ PiInv(Q_a))
q_b = (P @ PiInv(Q_b))
q_c = (P @ PiInv(Q_c))

Exercise 4.5

Define a function `estimateHomographies(Q_omega, qs)` which takes the following input arguments:
- `Q_omega`: an array original un-transformed checkerboard points in 3D, for example $Q_{\Omega}$.
- `qs`: a list of arrays, each element in the list containing $Q_{\Omega}$ projected to the image plane from different views, for example `qs` could be [$q_a$, $q_b$, $q_c$].

The function should return the homographies that map from $Q_{\Omega}$ to each of the entries in `qs`.

The homographies should work as follows:
$$ q = HQ_{\tilde{\Omega}} \tag{13} $$

where $Q_{\tilde{\Omega}}$ is $Q_{\Omega}$ without the z-coordinate, in homogeneous coordinates. Remember that we need multiple orientations of checkerboards e.g. rotated and translated.

Use your function `hest` from week 2 to estimate the individual homographies. You should return a list of homographies; one homography for each checkerboard orientation.

Test your function using $Q_{\Omega}$, $q_a$, $q_b$, and $q_c$. Check that the estimated homographies are correct with Equation 13.


In [134]:
Q_omega = checkerboard_points(10,20)

Q_a = Ra @ Q_omega 
Q_b = Rb @ Q_omega
Q_c = Rc @ Q_omega

q_a = Pi(P @ PiInv(Q_a))
q_b = Pi(P @ PiInv(Q_b))
q_c = Pi(P @ PiInv(Q_c))

Q_tilde = np.copy(Q_omega)
Q_tilde[-1,:] = 1 

H = hest(PiInv(q_a),Q_tilde)
estimated = H  @ Q_tilde  # H @ Q = s * q_a => H @ Q / q_a = s 


try:
    np.testing.assert_almost_equal((q_a), Pi(estimated))
    print("Success: The arrays are almost equal.")  # Or any other sign/notification of success.
except AssertionError as e:
    print("AssertionError:", e)





Success: The arrays are almost equal.


In [135]:
q_a = Pi(P @ PiInv(Q_a))
q_b = Pi(P @ PiInv(Q_b))
q_c = Pi(P @ PiInv(Q_c))

homographies = estimateHomograpies([PiInv(q_a),PiInv(q_b),PiInv(q_c)],Q_omega)
q_a_estimate = homographies[0] @ Q_tilde


try:
        
    np.testing.assert_almost_equal(q_a,Pi(homographies[0] @ (Q_tilde)))
    np.testing.assert_almost_equal(q_b,Pi(homographies[1] @ (Q_tilde)))
    np.testing.assert_almost_equal(q_c,Pi(homographies[2] @ (Q_tilde)))
    print("All the Homographies map correctly to the image matrix")
except AssertionError as e:
    print("The reprojected points are inequall to the original points", e)

All the Homographies map correctly to the image matrix


## Exercise 4.6

Now, define a function `estimate_b(Hs)` that takes a list of homographies `Hs` and returns the vector `b`. Form the matrix `V`. This is the coefficient matrix used to estimate `b` using SVD.

Test your function with the homographies from the previous exercise. See if you get the same result as by constructing $$B_{\text{true}} = K^{-T}K^{-1}$$, and converting this into `b_{true}`.

Is `b` a scaled version of `b_{true}`?

### Suggestions for debugging:
- Check that $$v_{11} \cdot b_{\text{true}} = h_1^T B_{\text{true}} h_1$$
- Be aware that $v_{αβ}$ use 1-indexing, while your code might not.


In [136]:
homographies = estimateHomograpies([q_a,q_b,q_c],Q_omega)
b = estimate_b(homographies)



B_true = np.linalg.inv(K_true.T) @ np.linalg.inv(K_true)
b_true = np.array([B_true[0,0], B_true[0,1], B_true[1,1], B_true[0,2], B_true[1,2], B_true[2,2]])

In [137]:
b,b_true

(array([-5.77350269e-01,  2.22044605e-16, -5.77350269e-01, -3.14405912e-15,
        -1.79100094e-15, -5.77350269e-01]),
 array([ 1.0000e-06,  0.0000e+00,  1.0000e-06, -9.6000e-04, -5.4000e-04,
         2.2132e+00]))

In [138]:
B = getB(b) 

v11 = estimate_b(homographies) 
left = v11 @ b_true
h1 = homographies[0]
right = h1[:,0] @ B @ h1[:,0].T


In [139]:
b_maybe = estimate_b_diagnostics(homographies)
b = estimate_b(homographies)

b_maybe,b

ze shape v12:(1, 6)
ze shape v12:(1, 6)
ze shape v12:(1, 6)
changed
shapezzz V: (6, 1, 6)


(array([[-0.0239138 , -0.95388655, -0.0239138 ,  0.20816969, -0.20816969,
          0.04782759],
        [-0.95388655,  0.11135142, -0.02227829,  0.1939326 , -0.1939326 ,
          0.04455658],
        [-0.0239138 , -0.02227829,  0.99944149,  0.00486186, -0.00486186,
          0.00111703],
        [ 0.20816969,  0.1939326 ,  0.00486186,  0.95767747,  0.04232253,
         -0.00972372],
        [-0.20816969, -0.1939326 , -0.00486186,  0.04232253,  0.95767747,
          0.00972372],
        [ 0.04782759,  0.04455658,  0.00111703, -0.00972372,  0.00972372,
          0.99776595]]),
 array([-5.77350269e-01,  2.22044605e-16, -5.77350269e-01, -3.14405912e-15,
        -1.79100094e-15, -5.77350269e-01]))

In [140]:
b_true

array([ 1.0000e-06,  0.0000e+00,  1.0000e-06, -9.6000e-04, -5.4000e-04,
        2.2132e+00])

In [141]:
get_CameraM(b_true*1).astype(int)

lambda:1.0
v_0:540.0
alpha:1000.0
beta:1000.0
gama:-0.0
u_0:960.0


array([[1000,    0,  960],
       [   0, 1000,  540],
       [   0,    0,    1]])

In [143]:
get_CameraM(b_maybe[0]).astype(int)

lambda:-0.04536786763028312
v_0:0.22384493588198615
alpha:1.3773677593376359
beta:nan
gama:nan
u_0:nan


/tmp/ipykernel_16646/1167550902.py:1: RuntimeWarning: invalid value encountered in cast
  get_CameraM(b_maybe[0]).astype(int)


array([[                   1, -9223372036854775808, -9223372036854775808],
       [                   0, -9223372036854775808,                    0],
       [                   0,                    0,                    1]])

In [ ]:
A = get_CameraM(b)


lambda:-0.5773502691895959
v_0:-3.102104627916144e-15
alpha:0.9999999999999614
beta:0.999999999999961
gama:3.84592537276688e-16
u_0:-5.445670138340628e-15


## Exercise 4.7

### Objective
Define a function `estimateIntrinsics(Hs)` that takes a list of homographies `Hs` and returns a camera matrix `K`.

### Instructions

- Utilize your `estimate_b` function from the previous exercise.
- From `b`, estimate the camera matrix `K` (referred to as `A` in the paper).
- Refer to Appendix B from the paper for the solution method.
- Test your function with the homographies provided in Exercise 4.5.

### Questions
- Do you retrieve the original camera matrix upon testing with the homographies from Exercise 4.5?

### Notes
Ensure to review the mathematical derivations in Appendix B of the paper to accurately implement the estimation of the camera matrix `K` based on `b`.


In [ ]:
K_est = estimateIntrisics(homographies)


K_est.astype(int)

lambda:-0.5773502691895959
v_0:-3.102104627916144e-15
alpha:0.9999999999999614
beta:0.999999999999961
gama:3.84592537276688e-16
u_0:-5.445670138340628e-15


array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 1]])

### Exercise 4.8

Define a function `Rs, ts = estimateExtrinsics(K, Hs)` that takes the camera matrix `K` and the homographies `Hs` and returns the rotations `Rs` and translations `ts` of each checkerboard. Utilize the formulas provided in the paper. Appendix C is not required for consideration — we can accept the associated error.

#### Considerations:
- The rotations `Rs` should not be identical to `Ra`, `Rb`, and `Rc` as the camera itself is also rotated.
- Reflect on what you expect the rotations to be. What kind of rotations do you observe, and are they valid?

#### Further Task:
Combine the functions to create a comprehensive function `K, Rs, ts = calibrateCamera(qs, Q)` that determines the camera intrinsics and extrinsics from the checkerboard correspondences `q` and `Q`.


In [ ]:
Rs,ts = estimateExtrinsics(K,homographies)


In [ ]:
get_CameraM(b_true).astype(int),get_CameraM(b_est*1000).astype(int)

lambda:1.0
v_0:540.0
alpha:1000.0
beta:1000.0
gama:-0.0
u_0:960.0


NameError: name 'b_est' is not defined

In [ ]:
Homographies = estimateHomograpies(Q_omega,[q_a,q_b,q_c])

R,T = estimateExtrinsics(K,Homographies)


In [ ]:
K_pred,Rs,ts = calibrateCamera([q_a,q_b,q_c],Q_tilde)
K= np.array([[f,0,dx],[0,f,dy],[0,0,1]])


lambda:0.5773502691896405
v_0:2.467840004883618e-15
alpha:1.0000000000000193
beta:1.000000000000019
gama:-0.0
u_0


### Exercise 4.9

Use your function to estimate K and R and t for each view. Use these to project Qω

thus reobtaining qa, qb and qc
, and verify that you get the same points by computing the reprojection
error.

In [ ]:
K,Rs,ts = calibrateCamera([q_a,q_b,q_c],Q_tilde)

P_a = K @ np.hstack((Rs[0],smart_transpose(ts[0]).T))
P_b = K @ np.hstack((Rs[1],smart_transpose(ts[1]).T))
P_c = K @ np.hstack((Rs[2],smart_transpose(ts[2]).T))


q_a_est = P_a @ PiInv(Q_tilde)
q_b_est = P_b @ PiInv(Q_tilde)
q_c_est = P_c @ PiInv(Q_tilde)


print(RMSE(q_a,q_a_est))
print(RMSE(q_b_est,q_b))
print(RMSE(q_c_est,q_c))

lambda:0.5773502691896405
v_0:2.467840004883618e-15
alpha:1.0000000000000193
beta:1.000000000000019
gama:-0.0
u_0
9.46753382489507
9.165151389911786
9.467533824895064
